# Press this Play button to Set Up the Models

In [1]:
from google.colab.output import eval_js

!git clone https://github.com/SarthakJaingit/Visually-Impaired-Food-Device-Aid-.git
%cd /content/Visually-Impaired-Food-Device-Aid-
!pip install effdet
!pip install timm

import webcam_backend 
import inference_device
import torch
import numpy as np
from PIL import Image
import io
import base64
import random
import cv2

def take_photo(label, img_data):
  data = eval_js('takePhoto("{}", "{}")'.format(label, img_data))
  return data

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = inference_device.load_torchvision_models("mobilenet_fasterrcnn", device)

def run_this():
    webcam_backend.start_input()
    label_html = 'Capturing...'
    img_data = ''
    count = 0

    color=None
    label=None
    line_thickness=None
    classes = ["Placeholder", "Apples", "Strawberry", "Peach", "Tomato", "Bad_Spots"]

    while True:
        js_reply = take_photo(label_html, img_data)
        if not js_reply:
            break
        
        image = webcam_backend.js_reply_to_image(js_reply)
        prediction = webcam_backend.webcam_infer_image(image, model, 0.15, [0.35, 0.1], device)

        drawing_array = np.zeros([512,512,4], dtype=np.uint8)

        for i, x in enumerate(prediction[0]['boxes']):
          tl = line_thickness or round(0.002 * (drawing_array.shape[0] + drawing_array.shape[1]) / 2) + 1  # line/font thickness
          color = color or [random.randint(0, 255) for _ in range(3)]
          c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
          cv2.rectangle(drawing_array, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
          label = classes[prediction[0]["labels"][i]]
          if label:
            tf = max(tl - 1, 1)  # font thickness
            t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
            c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
            cv2.rectangle(drawing_array, c1, c2, color, -1, cv2.LINE_AA)  # filled
            cv2.putText(drawing_array, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

        drawing_array[:,:,3] = (drawing_array.max(axis = 2) > 0 ).astype(int) * 255

        drawing_PIL = Image.fromarray(drawing_array, 'RGBA')
        iobuf = io.BytesIO()
        drawing_PIL.save(iobuf, format='png')
        drawing_bytes = 'data:image/png;base64,{}'.format((str(base64.b64encode(iobuf.getvalue()), 'utf-8')))

        img_data = drawing_bytes


Cloning into 'Visually-Impaired-Food-Device-Aid-'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 210 (delta 99), reused 142 (delta 43), pack-reused 0
Receiving objects: 100% (210/210), 123.65 MiB | 26.83 MiB/s, done.
Resolving deltas: 100% (99/99), done.
Checking out files: 100% (22/22), done.
/content/Visually-Impaired-Food-Device-Aid-
     |████████████████████████████████| 111 kB 5.4 MB/s 
     |████████████████████████████████| 74 kB 2.9 MB/s 
     |████████████████████████████████| 376 kB 23.9 MB/s 
     |████████████████████████████████| 636 kB 33.2 MB/s 
     |████████████████████████████████| 112 kB 36.7 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=0a786c2261495feb61e2c9f78a0e7f571c344a5b2962d5798c9070a8fcf4e4b5
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abc

Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_320_fpn-907ea3f9.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_320_fpn-907ea3f9.pth


  0%|          | 0.00/74.2M [00:00<?, ?B/s]

Loaded model weights for mobilenet_fasterrcnn turning to eval model


# Run this cell to use webcam

Note: For best results do the following: 
* Hold fruits relatively still close to camera and wait for like 2-3 seconds till the model makes continual predictions 
* If you move the object around fast expect a noticeable delay in the drawing of boxes on the frame.


In [2]:
run_this()

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
